## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Layer Convolutional:
* [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)(in_channels, out_channels, kernel_size, stride=1, padding=0)

Layere Pooling:
* [torch.nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)(kernel_size, stride=None, padding=0)
*  [torch.nn.AveragePool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html)(kernel_size, stride=None, padding=0)

Layere Adaptive Pool, intalnit adesea si ca Global Pool:
* [torch.nn.AdaptiveAvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html)(output_size)
* [torch.nn.AdaptiveMaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveMaxPool2d.html)(output_size)

Layer de liniarizare:

* [torch.nn.Flatten()](https://pytorch.org/docs/stable/generated/torch.flatten.html)



Four **hyperparameters** control the size of the output volume:
* **Depth**: number of filters, as each filter _looks_ at different areas of the input:
* **Stride**: the step taken when _sliding_ the filter. (Usually 1 or 2, 3 - uncommon).
* **Zero-Padding**: size of the number of 0s that surround the border of the input volume. Example: If you want to the same width and height for input and output.
* **Dilation**: Distance between elements of the convolutional kernel.



**Why Pooling Layer?**

1. Modifica volumul de input (input volume) in reprezentari _mai mici_ si mai usor de _manevrat_.
2. Opereaza independent pe fiecare Activation Map.

<img src="https://computersciencewiki.org/images/9/9e/MaxpoolSample.png" width="425" height="300"/> <img src="https://miro.medium.com/v2/resize:fit:517/0*lIcR0gOMK0xzTr6_.png" width="425" height="300"/>






**Why Adaptive Pooling Layer?**

1. Folosite de regula in etapele finale de constructie a unei arhitecturi de tipul _ConvNet_ pentru a inlocui fully-connected layers.
2. Incearca sa previna *overfitting phenomenon* fortand feature maps sa retina informatia **globala** care este relevanta pentru task-ul acestei _ConvNet_ (clasificare, identifcare etc.)

<img src="https://www.researchgate.net/publication/338079465/figure/fig4/AS:905983672987648@1593014748012/The-difference-of-max-pooling-and-global-max-pooling.ppm" width="725" height="300"/>


<img src="https://drive.google.com/uc?id=11l7Xsh-iQmASvXTkgH2MgtA01XCW6CAC">


[Visualise them Here](https://github.com/vdumoulin/conv_arithmetic).

In [ ]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 25, 25] # Stride & Padding
*   [1, 10, 32, 32]
*  [1, 3, 2, 2]



In [ ]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

## Instantierea seturilor de date

In [ ]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __collate_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *collate_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

def collate_fn(examples):
  ### Completati codul pentru cerinta aici
  return examples


train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori (0.25 p).
    * Liniarizati iesirea din cel de-al doilea strat convolutional (0.25 p).
    * Adaugat stratul final de tipul `fully-connected` (0.25 p).
    * Folositi o functie de activare la alegere (Exemplu [RELU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)) (0.25 p).

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [ ]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    ### Completati codul pentru cerinta aici
    pass

  def forward(self,x):
    ### Completati codul pentru cerinta aici
    return x

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Numarul de epoci (0.25 p)
  * Retea (0.25 p)
  * Optimizator (0.25 p)
  * Alegeti functia de cost (0.25 p)

In [ ]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = None

# Definiti reteaua
network = None

# Definiti optimizatorul
optimizer = None
"""
Dupa definirea optimizatorului si dupa fiecare iteratie de antrenare, trebuie
apelata functia zero_grad() pentru a seta valoare tuturor gradientilor la zero.
"""
# Completati aici codul pentru seta valoare tuturor gradientilor la zero


# Definiti functia cost pentru clasificare Cross-Entropy
# https://neptune.ai/blog/pytorch-loss-functions
loss_fn = None

## Definirea functiei de antrenare

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
def test_acc(net: nn.Module, test_loader: DataLoader):
  net.eval()

  total = 0
  correct = 0

  for test_images, test_labels in test_loader:
    total += len(test_images)
    out_class = torch.argmax(net(test_images))
    correct += torch.sum(out_class == test_labels)

  return correct / total * 100


def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader,
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    net.train()

    # Iteram prin fiecare batch din dataloader
    for images, labels in train_loader:
      # Aplicam reteaua neurala pe imaginile din batch-ul curent
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe etichetele imaginilor
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a actualiza parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()

    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Calculam acuratetea
    acc = test_acc(net, test_loader)
    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e + 1, acc))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [ ]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

## Reteaua LeNet

### Cerinte
  * **(3p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o.



![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet

_Question:_ Care este diferenta dintre `tanh` si `softmax`? De ce credeti ca peste ultimul layer (cel de output) a fost aplicata functia `softmax`?


In [ ]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    ### Completati codul pentru cerinta aici
    pass

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    ### Completati codul pentru cerinta aici
    return x

## Optional: Reteaua AlexNet ❤️

❗Daca alegeti aceasta retea veti continua sa rezolvati exercitiile urmatoare pentru reteaua AlexNet.

Pentru a usura volumul de munca si obtine o retea AlexNet comparabila in dificultate cu LeNet, urmati acesti pasi:

✔️ Includeti functii de activare intre layere (exemplu ReLU).

✔️ Va folositi doar de prima subsectiune din schema figurii arhitecturii AlexNet (adica doar Conv1 si Conv2 blocks).

✔️ Inputul vostru se opreste la un minimum size de 8x8.

✔️ Modificati output-ul retelei sa prezica 10 clase in loc de 1000 de clase.





![alt text](https://anhreynolds.com/img/alexnet.png)

Figura arhitectura AlexNet.

![alt text](https://anhreynolds.com/img/alexnet-parameters.png)

Tabel arhitectura AlexNet


In [ ]:
import torch.nn as nn

class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    pass

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [ ]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = None

# Definiti reteaua
lenet = None

# Definiti optimizatorul
lenet_optimizer = None
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = None

## Antrenarea retelei LeNet

In [ ]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* [torch.nn.BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) (num_features)
* [torch.nn.InstanceNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.InstanceNorm2d.html) (num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de alte functii de activare:

* ReLU
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme, pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.

**Punctaj:** 0.6p / configuratie.

0.6p din care:
- 0.4p modificarea retelei
- 0.1p obtinerea rezultatelor
- 0.1p afisarea acestora si explicatie.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  
